# Use RASA DSL to Configure DeepPavlov's GO-Bot

At DeepPavlov, we support a variety of industry-wide and popular standards to support developing Conversational AI solutions.
DSLs, known as Domain-Specific Languages, provide a rich mechanism to define the behavior, or "the what", while
the underlying system uses the parser to transform these definitions into commands that implement this behavior, or "the how"
using the system's components.
Until very recently we supported two such DSLs, including industry-standard [AIML](http://docs.deeppavlov.ai/en/master/features/skills/aiml_skill.html), as well as [DSL](http://docs.deeppavlov.ai/en/master/features/skills/dsl_skill.html) designed by one our partners, EORA.

In this tutorial, you will learn how to use another industrial DSL, or, better said, set of DSLs, introduced by RASA.ai,
to build simple goal-oriented chatbots using DeepPavlov's GO-bot.

As discussed in our [blog post](todo/link), this is the very beginning of our work focused on supporting RASA DSLs as a way to configure DeepPavlov-based
goal-oriented chatbots, and therefore not all elements of the RASA DSLs are supported. It is also worth mentioning that in this release we want to focus on supporting tools 
to define the domain logic behind the goal-oriented assistant, and files like `config.yml` and others are out of scope for this release.

To configure a DeepPavlov-based goal-oriented chatbot using these DSLs, you need to have at least three basic config files:
* `stories.md` (or `stories-{trn, tst, val}.md` but these are just subsamples)
* `nlu.md`
* `domain.yml`

These files allow you to define 3 key elements of the chatbot, including product-level stories, NLU training data, and your chatbot's domain.

## Concepts Behind Stories.md, NLU.md, and Domain.yml

### `stories.md`



`stories.md` is a mechanism used to teach your chatbot how to respond to user messages. It allows you to control your chatbot's dialog management.

These "stories" model real conversations between a user and a chatbot. This Markdown-based file is used to define a list of
*stories*, and each *story* can have a list of one or more *intents* with (optional) corresponding *slots*, where each *intent*
has one or more corresponding actions taken by the chatbot.

These actions, in general, can be anything, from simple message replies, to programmable actions that call APIs of other services.
*Note:* In this version, supported actions are limited to simple message replies.

In a way, it can be seen as a *dialogues dataset*.

#### format

Stories file is a markdown file of the following format:

```markdown
## story_title(not used by algorithm, but useful to work with for humans)
* user_action_label{"1st_slot_present_in_action": "slot1_value", .., "Nth_slot_present_in_action": "slotN_value"}
 - system_respective_utterance
* another_user_action_of_the_same_format
  - another_system_response
...

## another_story_title
...

```

**See examples below in this tutorial**

### `nlu.md`


`nlu.md` represents an NLU model of your chatbot. It allows you to provide training examples that show how your chatbot should
understand user messages, and then train a model through these examples.

While DeepPavlov's GO-bot supports JSON-based DSTC-2 format for training data, this Markdown Format introduced by RASA is the easiest one for humans to read and write.

#### format

NLU file is a markdown file of the following format:

```markdown
## intent:possible_user_action_label_1
- An example of user text that has the possible_user_action_label_1 action label
- Another example of user text that has the possible_user_action_label_1 action label
...

## intent:possible_user_action_label_N
- An example of user text that has the (possible_user_action_label_N)[action_label] action label
<!-- Slotfilling dataset is provided as an inline markup of user texts -->
...

```

**See examples below in this tutorial**

### `domain.yml`

`domain.yml` helps you to define the universe your chatbot lives in: what user inputs it expects to get, what actions it should be able to predict,
how to respond, and what information to store.
This YML format is relatively simple, and it can be seen as a dictionary of all components of your chatbot, including but not limited to intents,
actions, responses, and other things.

#### format

Domain file is a YAML file of the following format:

```yaml
# slots section lists the possible slot names (aka slot types) 
# that are used in the domain (i.e. relevant for bot's tasks)
# currently only type: text is supported
slots:
  slot1_name:
    type: text
  ...
  slotN_name:
    type: text

# entities list now follows the slots list 2nd level keys 
# and is present to support upcoming features. Stay tuned for updates with this!
entities:
- slot1_name
...
- slotN_name

# intents section lists the intents that can appear in the stories
# being kept together they do describe the user-side part of go-bot's experience
intents:
  - user_action_label
  - another_user_action_of_the_same_format
  ...

# responses section lists the system response templates.
# Despite system response' titles being usually informative themselves
#   (one could even find them more appropriate when no actual "Natural Language" is needed 
#    (e.g. for buttons actions in bot apps))
# It is though extremely useful to be able to serialize the response title to text. 
# That's what this section content is needed for.
responses:
  system_utterance_1:
    - text: "The text that system responds with"
  another_system_response:
    - text: "Here some text again"

```

**See examples below in this tutorial**

## Basic Chatbot

Let's build the simplest chatbot possible.
This chatbot will be capable of processing three intents: *greeting*, *goodbye*, and *thanks*.

In [1]:
DP_MIN_DEMO_DIR = "dp_minimal_demo_dir"  # we will work in this folder

In [2]:
import os

%cd /content
os.makedirs(DP_MIN_DEMO_DIR, exist_ok=True)
%cd {DP_MIN_DEMO_DIR}

/content
/content/dp_minimal_demo_dir


### Stories.md: Basic Stories Example

`stories.md` is pretty straightforward in this case. In it you define 3 stories, each having its own intent and response (utterance).
Take into account the fact that you can combine all of these intents under one story, or add two intents to one stories, and third to another one.

In [3]:
%%writefile stories.md

## greet
* greet
  - utter_greet

## thank
* thank
  - utter_noworries

## goodbye
* bye
  - utter_bye

Writing stories.md


### nlu.md: Basic NLU Training Data Example

`nlu.md` has an NLU training data that enables DeepPavlov to recognize user phrases as belonging to one the pre-defined intents.

In [4]:
%%writefile nlu.md

## intent:greet
- Hi
- Hey
- Hi bot
- Hey bot
- Hello
- Good morning
- hi again
- hi folks

## intent:bye
- goodbye
- goodnight
- good bye
- good night
- see ya
- toodle-oo
- bye bye
- gotta go
- farewell

## intent:thank
- Thanks
- Thank you
- Thank you so much
- Thanks bot
- Thanks for that
- cheers


Writing nlu.md


### domain.yml: Basic Domain Example

`domain.yml` lists the list of: 
* possible user action intents 
* possible system response actions

*Note:* Entities and slots are omitted in this example. See the more sophisticated example below to see how they can be defined in the `domain.yml`.

In [5]:
%%writefile domain.yml

intents:
  - greet
  - bye
  - thank

responses:
  utter_noworries:
  - text: No worries!
  utter_greet:
  - text: Hi
  utter_bye:
  - text: Bye!

Writing domain.yml


The next step is to install the `deeppavlov` library.

In [6]:
!pip install git+https://github.com/deepmipt/DeepPavlov.git@feature/gobot-md-yaml-config
!python -m deeppavlov install gobot_simple_dstc2

  Cloning https://github.com/deepmipt/DeepPavlov.git (to revision feature/gobot-md-yaml-config) to /tmp/pip-req-build-h4qrt5dz
  Running command git clone -q https://github.com/deepmipt/DeepPavlov.git /tmp/pip-req-build-h4qrt5dz
  Running command git checkout -b feature/gobot-md-yaml-config --track origin/feature/gobot-md-yaml-config
  Switched to a new branch 'feature/gobot-md-yaml-config'
  Branch 'feature/gobot-md-yaml-config' set up to track remote branch 'feature/gobot-md-yaml-config' from 'origin'.
     |████████████████████████████████| 51kB 2.6MB/s 
     |████████████████████████████████| 2.1MB 9.6MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
     |████████████████████████████████| 1.5MB 34.9MB/s 
     |████████████████████████████████| 20.1MB 61.6MB/s 
     |████████████████████████████████| 10.4MB 45.2MB/s 
     |████████████████████████████████| 512kB 42.8MB/s 
     |████████████████████████████████| 7.3MB 31.5MB/s 
     |████████████████████████████████| 51kB 

2020-08-04 05:55:52.96 INFO in 'deeppavlov.core.common.file'['file'] at line 32: Interpreting 'gobot_simple_dstc2' as '/usr/local/lib/python3.6/dist-packages/deeppavlov/configs/go_bot/gobot_simple_dstc2.json'
     |████████████████████████████████| 24.2MB 127kB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 10.4MB 4.2MB/s 
     |████████████████████████████████| 2.2MB 38.9MB/s 
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
     |████████████████████████████████| 696kB 4.1MB/s 
     |████████████████████████████████| 110.5MB 89kB/s 
     |████████████████████████████████| 512kB 39.0MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 3.8MB 37.0M

Define the path to our DSL-based configuration files above (the folder we are in right now) and the folder used to store the trained bot.

In [7]:

from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)

gobot_config['metadata']['variables']['DATA_PATH'] = '.'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

Since our data is basically the mock tutorial data we will use the same subsamples for all of the train (training set), test (test set) and valid (validation set) subsamples. 

However, for a real system you should use different train, test, and valid sample stories.md files.

In [8]:
!cp stories.md stories-trn.md
!cp stories.md stories-tst.md 
!cp stories.md stories-val.md 

The next step is to train the bot:

In [9]:
from deeppavlov import train_model

train_model(gobot_config, download=True)

2020-08-04 05:57:01.557 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/datasets/gobot_md_yaml_minimal.tar.gz to /content/gobot_md_yaml_minimal.tar.gz
100%|██████████| 528/528 [00:00<00:00, 216kB/s]
2020-08-04 05:57:01.935 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /content/gobot_md_yaml_minimal.tar.gz archive into /content/dp_minimal_demo_dir
2020-08-04 05:57:02.315 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt to /root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt
347MB [00:34, 10.1MB/s]
2020-08-04 05:57:37.80 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=False, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge objec

2020-08-04 05:57:41.229 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f5236829128>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slots': []}, {'text': 'toodle-oo', 'slots': []}, {'text': 'bye bye', 'slots': []}, {'text': 'gotta go', 'slots': []}, {'text': 'farewell', 'slots': []}]}, 'thank': {(





Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.

Instructions for updating:
Use standard file APIs to check for files with this prefix.


2020-08-04 05:58:22.690 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 89: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from scratch.
2020-08-04 05:58:22.761 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 198: Initial best per_item_action_accuracy of 0.1667


{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 0.1667}, "time_spent": "0:00:01", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}


...

2020-08-04 05:59:05.704 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-08-04 05:59:05.709 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f522ee64438>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slot

{"valid": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}
{"test": {"eval_examples_count": 3, "metrics": {"per_item_action_accuracy": 1.0}, "time_spent": "0:00:01"}}


2020-08-04 05:59:05.711 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /tmp/tmpzeq5chss]
2020-08-04 05:59:05.715 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 306: AFTER MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f522ee64438>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slots': []}, {'text': 't

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


Chainer[<deeppavlov.models.go_bot.wrapper.DialogComponentWrapper at 0x7f5236829908>,

Finally, it's time to build our bot and experiment with it:

In [10]:
from deeppavlov import build_model
bot = build_model(gobot_config)

2020-08-04 05:59:46.632 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /content/dp_minimal_demo_dir/word.dict]
2020-08-04 05:59:46.641 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 217: BEFORE MLU_MD_DialogsDatasetReader._read_story(): story_fpath=/content/dp_minimal_demo_dir/stories-trn.md, dialogs=True, domain_knowledge=<deeppavlov.dataset_readers.md_yaml_dialogs_reader.DomainKnowledge object at 0x7f5225c6c8d0>, intent2slots2text={'greet': {(): [{'text': 'Hi', 'slots': []}, {'text': 'Hey', 'slots': []}, {'text': 'Hi bot', 'slots': []}, {'text': 'Hey bot', 'slots': []}, {'text': 'Hello', 'slots': []}, {'text': 'Good morning', 'slots': []}, {'text': 'hi again', 'slots': []}, {'text': 'hi folks', 'slots': []}]}, 'bye': {(): [{'text': 'goodbye', 'slots': []}, {'text': 'goodnight', 'slots': []}, {'text': 'good bye', 'slots': []}, {'text': 'good night', 'slots': []}, {'text': 'see ya', 'slot

INFO:tensorflow:Restoring parameters from /content/dp_minimal_demo_dir/model/policy


In [11]:
bot.reset()

bot(["start"])
bot(["Hi"])[0][0].actions_tuple

('utter_greet',)

Our bot answers with "greeting" to our "greeting". How will it respond to some grateful message?

In [12]:
bot.reset()
bot(["start"])

bot(["Thanks!"])[0][0].actions_tuple

('utter_noworries',)

Ok, "no worries" is an expected response. Let's check if the "goodbye" user message is processed with the corresponding reply:

In [13]:
bot.reset()
bot(["start"])

bot_response_actions = bot(["bye"])[0][0].actions_tuple

In [14]:
import yaml

system_utter2text = yaml.load(open("domain.yml"))["responses"]
system_utter2text[bot_response_actions[0]][0]["text"]

'Bye!'

## Advanced Demo: Building a Restaurants Bot inspired by the DSTC Schema-Guided Dialogue Dataset

In [15]:
DP_BIG_DEMO_DIR = "dp_big_demo_dir"  # we'll work in this directory

In [16]:
import os

%cd /content
os.makedirs(DP_BIG_DEMO_DIR, exist_ok=True)
%cd {DP_BIG_DEMO_DIR}

/content
/content/dp_big_demo_dir


While the previous demo was focused on figuring out how to work with the very simple goal-oriented chatbot, the reality of chatbots is rarely that simple. 
Take, for example, the use case for restaurants. People can search for them, ask about the menus, or book tables. These activities require a substancially 
more advanced configuration.

In the purpose of this rather more relastic demo, we decided to go through a rather unusual route. To simplify the process of defining the domain and behavior 
of this chatbot, we took a famous industrial research dataset provided by Google Research known as DSTC, or [Schema Dataset](https://github.com/google-research-datasets/dstc8-schema-guided-dialogue).

This dataset contains a huge number of the annotated human-machine conversations 
crowdsourced in an [M2M manner](https://arxiv.org/pdf/1801.04871.pdf) for various real-life scenarios and domains.

One of these domains is dedicated to *Restaurants*. In it, users are performing a variety of the goal-oriented tasks like searching for restaurants or booking tables via interaction with the bot.

Given the power and elegance of the DSTC format, we took liberty to use our **automatic conversion tool** to directly **transform** its data into the set of _stories.md_, _nlu.md_, _domain.yml_. 

##### Download the data used in this tutorial section

In [17]:
# let's get the mentioned converted Schema-dataset subset
!wget http://files.deeppavlov.ai/datasets/schema_resto_md_yaml_v2.tar.gz
!tar -zxf schema_resto_md_yaml_v2.tar.gz 

--2020-08-04 06:00:28--  http://files.deeppavlov.ai/datasets/schema_resto_md_yaml_v2.tar.gz
Resolving files.deeppavlov.ai (files.deeppavlov.ai)... 93.175.29.74
Connecting to files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 100665 (98K) [application/octet-stream]
Saving to: ‘schema_resto_md_yaml_v2.tar.gz’

schema_resto_md_yam 100%[===================>]  98.31K   185KB/s    in 0.5s    

2020-08-04 06:00:29 (185 KB/s) - ‘schema_resto_md_yaml_v2.tar.gz’ saved [100665/100665]



#### Technical Note: Automatic Conversion from DSTC Schema Format to RASA DSLs 

Schema dataset is provided in DSTC (dialogue state tracking challenge) challenge [format](https://github.com/google-research-datasets/dstc8-schema-guided-dialogue).

The DSTC format has its own advantages: it is very detailed and allows for various additional info to be incorporated to the dataset itself. 

In it, there are two major components - Schema Representation, and Dialog Representation. The first component is dedicated to describing sets of Intents, Slots, and Entities that are used by a given service through an API. the second component is focused on describing actual dialogs that happen between users and services. It also includes actual labels for the aforementioned Intents, Slots, and Entities defined in the Schema component. 

However, while DSTC format is quite rich for building state-of-the-art systems that participate in the annual DSTC competitions, it takes a serious effort for the real-world developers to collect and annotate data in this format. In contrast, RASA DSLs we're illustrating here are quite different from the DSTC: they are meant to be neat and minimalistic, and to allow developers to define their domains from a rather scarce input information. 

As mentioned in the beginning of this part of the tutorial, we've performed an automatical conversion of the Schema Restaurants dataset from the DSTC format to RASA DSL.





#### Slot Filler

Any typical goal-oriented chatbot system uses a standard approach to define the way it works in the form of a pipeline. DeepPavlov's Go-Bot is quite permissive in which components
can be used in it's pipeline; however, Slot Filler is the required one.

Slot Filler, also known as slotfiller, is necessary to recognize and normalize slot-filling information provides in user's utterances.

For example, when user says that she wants to "book a table *in London*", slotfiller's job is to recognize that *London* in this phrase represents the required slot `city`.

For the purposes of this demo, we are providing the pretrained slotfiller for the dataset. See [the small notebook](todo/link) on how the slotfiller was trained.

In [18]:
from deeppavlov import configs, train_model, build_model
from deeppavlov.core.common.file import read_json

!python -m deeppavlov download schema_resto_md_yaml/ner_config.json
!python -m deeppavlov download schema_resto_md_yaml/slotfiller_config.json

slotfill_config = read_json("schema_resto_md_yaml/slotfiller_config.json")
slotfiller = build_model(slotfill_config, download=True)


2020-08-04 06:00:43.9 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/datasets/schema_resto_md_yaml_v2.tar.gz?config=ner_config to /root/.deeppavlov/schema_resto_md_yaml_v2.tar.gz
100% 101k/101k [00:00<00:00, 286kB/s] 
2020-08-04 06:00:43.720 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/schema_resto_md_yaml_v2.tar.gz archive into /root/.deeppavlov/downloads
2020-08-04 06:00:44.91 INFO in 'deeppavlov.core.data.utils'['utils'] at line 94: Downloading from http://files.deeppavlov.ai/deeppavlov_data/schema_ner.tar.gz?config=ner_config to /root/.deeppavlov/schema_ner.tar.gz
100% 1.67M/1.67M [00:01<00:00, 1.32MB/s]
2020-08-04 06:00:45.727 INFO in 'deeppavlov.core.data.utils'['utils'] at line 269: Extracting /root/.deeppavlov/schema_ner.tar.gz archive into /root/.deeppavlov/models
2020-08-04 06:00:46.789 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embe

2020-08-04 06:00:54.160 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/embeddings/glove.6B.100d.txt download because of matching hashes
2020-08-04 06:00:54.530 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/datasets/schema_resto_md_yaml_v2.tar.gz download because of matching hashes
2020-08-04 06:00:54.908 INFO in 'deeppavlov.download'['download'] at line 132: Skipped http://files.deeppavlov.ai/deeppavlov_data/schema_ner.tar.gz download because of matching hashes
2020-08-04 06:00:55.290 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/schema_ner/word.dict]
2020-08-04 06:00:55.299 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading vocabulary from /root/.deeppavlov/models/schema_ner/tag.dict]
2020-08-04 06:00:55.306 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 115: [loading voca


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Use `tf.cast` instead.



2020-08-04 06:01:36.332 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 


The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
seq_dim is deprecated, use seq_axis instead
Instructions for updating:
batch_dim is deprecated, use batch_axis instead


2020-08-04 06:01:36.435 INFO in 'deeppavlov.core.layers.tf_layers'['tf_layers'] at line 760: 
Warning! tf.contrib.cudnn_rnn.CudnnCompatibleLSTMCell is used. It is okay for inference mode, but if you train your model with this cell it could NOT be used with tf.contrib.cudnn_rnn.CudnnLSTMCell later. 
2020-08-04 06:01:38.156 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /root/.deeppavlov/models/schema_ner/model_no_pos]


INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/schema_ner/model_no_pos


2020-08-04 06:01:38.306 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/root/.deeppavlov/downloads/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.


In [19]:
slotfiller(["i'm looking for a thai food somewhere in SFO"])

[{'city': 'SFO', 'cuisine': 'Thai'}]

Seems OK. Let's save our slotfiller config to train and evaluate the restaurants bot, finally.

In [20]:
import json

json.dump(slotfill_config, open('slotfill_config.json', 'wt'))

#### Known Limitations

While slotfilling technology uses the power of the industry-standard Named Entity Recognition (NER) method to recognize key slots in the given phrases, 
the quality of slot recognition could be substantially increased by combining this process with the data already known to you as the developer. 
For example, having a finite list of cities that are supported by a given end-user solution (e.g., several cities in Greater Seattle Area for local restaurant chain) 
aids significantly to the slotfiller. Typically, this information is stored in the database, though it could also be available in loose files like CSV (comma-separated values).

However, in order to focus on the support of the RASA DSLs, we made a decision to omit the support for such data. An additional demo will be provided in consequent releases.

Nevertheless our demo goal-oriented bot should still be able to generalize and use the global patterns in the conversations. 

### Stories.md: Advanced Stories Example

`stories.md`

*Note: As said above, this file has been auto-generated from the DSTC schema.*

Like in the Basic Demo, Stories here define a variety of interactions between user and our bot.

In [21]:
!head -500 schema_resto_md_yaml/stories.md | tail -30

  - utter_INFORM_Cuisine+INFORM_StreetAddress+NOTIFY_SUCCESS
* THANK_YOU+GOODBYE{}
  - utter_GOODBYE

## 25
* hi{}
  - utter_hi
* INFORM_INTENT_Intent{"intent": "FindRestaurants"}
  - utter_REQUEST_City
* INFORM_City{"city": "Oakland"}
  - utter_REQUEST_Cuisine
* INFORM_Cuisine{"cuisine": "Fish"}
  - utter_OFFER_RestaurantName+OFFER_City+INFORM_COUNT_Count
* REQUEST_ALTS{}
  - utter_OFFER_RestaurantName+OFFER_City
* INFORM_PriceRange+REQUEST_ALTS{"price_range": "moderate"}
  - utter_OFFER_RestaurantName+OFFER_City+INFORM_COUNT_Count
* REQUEST_ServesAlcohol+REQUEST_HasLiveMusic{}
  - utter_INFORM_HasLiveMusic+INFORM_ServesAlcohol
* INFORM_INTENT_Intent+SELECT{"intent": "ReserveRestaurant"}
  - utter_REQUEST_Time
* INFORM_PartySize+INFORM_Time{"party_size": "2", "time": "10:30 in the morning"}
  - utter_CONFIRM_RestaurantName+CONFIRM_City+CONFIRM_Time+CONFIRM_PartySize+CONFIRM_Date
* REQUEST_StreetAddress+REQUEST_PhoneNumber+AFFIRM{}
  - utter_NOTIFY_FAILURE+REQ_MORE
* INFORM_PartySize+I

### nlu.md: Advanced NLU Training Data Example

`nlu.md`

*Note: As said above, this file has been auto-generated from the DSTC schema.*

Like in the Basic Demo, `nlu.md` shows the examples of user utterances for the supported intent classes.
The slotfilling and NER information is provided in the form of the inline mark-up.

In [22]:
!head -50 schema_resto_md_yaml/nlu.md | tail -20

- [Evening 7:15](time) at Mcdonald's.
- [6:15 in the evening](time) at Bistro liaison.
- Let's set it up for [6:30 pm](time) at Sakoon.
- I would like a reservation in [Flames Eatery](restaurant_name) at [19:00](time)
- I'm thinking about Rice Bowl, at [morning 11:30](time)

## intent:REQUEST_Cuisine+AFFIRM
- Sounds great. Do you know if they cook [Taiwanese](cuisine) food?
- yeah. what do they serve?

## intent:AFFIRM
- Yes
- That's great.
- Sure that sounds good.
- That works!
- that would be great.
- That sounds good.
- Yes that works for me
- Sure, that sounds great!
- This is good for me


### domain.yml: Advanced Domain Example

`domain.yml`

*Note: As said above, this file has been auto-generated from the DSTC schema.*

The domain file now provides the list of slots and entities relevant for the bot as well as the supported intent classes and system response text templates.

In [23]:
!head -40 schema_resto_md_yaml/domain.yml && echo "..."
!grep -B 1 -A 10 responses schema_resto_md_yaml/domain.yml && echo "..."

slots:
  price_range:
    type: text
  cuisine:
    type: text
  date:
    type: text
  restaurant_name:
    type: text
  intent:
    type: text
  party_size:
    type: text
  time:
    type: text
  has_live_music:
    type: text
  city:
    type: text
  serves_alcohol:
    type: text

entities:
- price_range
- cuisine
- date
- restaurant_name
- intent
- party_size
- time
- has_live_music
- city
- serves_alcohol

intents:
  - INFORM_RestaurantName+INFORM_Time
  - THANK_YOU+GOODBYE
  - INFORM_Time+INFORM_RestaurantName
  - INFORM_PriceRange+REQUEST_ALTS
  - REQUEST_PhoneNumber+REQUEST_StreetAddress+AFFIRM
...

responses:
  utter_REQUEST_Cuisine:
    - text: "What are you in the mood for? {cuisine}? {cuisine}"
  utter_REQUEST_City:
    - text: "What city are you interested in?"
  utter_OFFER_RestaurantName+OFFER_City:
    - text: "Great! {restaurant_name} is a great restaurant in {city}"
  utter_INFORM_PhoneNumber+INFORM_HasLiveMusic:
    - text: "The contact number is {phone_number}"
  

Now that we have all three key files, like in the Basic Demo, we can now proceed with our bot's training.

In [24]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json


gobot_config = read_json(configs.go_bot.gobot_md_yaml_minimal)
gobot_config['chainer']['pipe'][-1]['slot_filler'] = {"config_path": "slotfill_config.json"}

gobot_config['metadata']['variables']['DATA_PATH'] = 'schema_resto_md_yaml'
gobot_config['metadata']['variables']['MODEL_PATH'] = '.'

Since our data is the tutorial data we will use the same subsamples for all of train (training set), test (test set), and valid (validation set) subsamples.

However, for a real system you should use different train, test, and valid sample stories.md files.

In [25]:
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-trn.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-tst.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-val.md

In [26]:
from deeppavlov import train_model

train_model(gobot_config, download=False);

2020-08-04 06:07:15.332 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 211: Did not improve on the per_item_action_accuracy of 0.695


{"valid": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.6842}, "time_spent": "0:03:57", "epochs_done": 14, "batches_seen": 315, "train_examples_seen": 1218, "impatience": 2, "patience_limit": 10}}
{"train": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.6782}, "time_spent": "0:04:04", "epochs_done": 14, "batches_seen": 330, "train_examples_seen": 1275, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.5178457121054332}}


2020-08-04 06:07:26.748 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 211: Did not improve on the per_item_action_accuracy of 0.695


{"valid": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.6794}, "time_spent": "0:04:08", "epochs_done": 14, "batches_seen": 330, "train_examples_seen": 1275, "impatience": 3, "patience_limit": 10}}
{"train": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.6926}, "time_spent": "0:04:15", "epochs_done": 15, "batches_seen": 345, "train_examples_seen": 1335, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.4527850568294525}}


2020-08-04 06:07:37.983 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 211: Did not improve on the per_item_action_accuracy of 0.695


...

{"test": {"eval_examples_count": 85, "metrics": {"per_item_action_accuracy": 0.7536}, "time_spent": "0:00:05"}}


2020-08-04 06:17:29.880 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 260: INSIDE MLU_MD_DialogsDatasetReader._read_story(): Skipping story w. line * INFORM_City+INFORM_PriceRange+INFORM_Cuisine{"city": "Mountain View", "price_range": "moderate", "cuisine": "French"} because of no NLU candidates found
2020-08-04 06:17:29.883 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 260: INSIDE MLU_MD_DialogsDatasetReader._read_story(): Skipping story w. line * INFORM_PriceRange+INFORM_HasLiveMusic+INFORM_Cuisine{"price_range": "inexpensive", "has_live_music": "True", "cuisine": "salad"} because of no NLU candidates found
2020-08-04 06:17:29.885 DEBUG in 'deeppavlov.dataset_readers.md_yaml_dialogs_reader'['md_yaml_dialogs_reader'] at line 260: INSIDE MLU_MD_DialogsDatasetReader._read_story(): Skipping story w. line * INFORM_PartySize+NEGATE{"party_size": "1"} because of no NLU candidates found
2020-08-

INFO:tensorflow:Restoring parameters from /root/.deeppavlov/models/schema_ner/model_no_pos


2020-08-04 06:18:13.785 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 45: Load path '/root/.deeppavlov/downloads/schema_resto_md_yaml/slotfill.json' differs from save path '/root/.deeppavlov/models/slotfill_dstc2/model' in 'infer' mode for DstcSlotFillingNetwork.
2020-08-04 06:18:13.790 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `/root/.deeppavlov/downloads/embeddings/glove.6B.100d.txt`]
2020-08-04 06:18:53.651 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-08-04 06:18:53.657 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [27]:
bot = build_model(gobot_config)

2020-08-04 06:20:18.250 INFO in 'deeppavlov.models.go_bot.policy.policy_network'['policy_network'] at line 86: INSIDE PolicyNetwork init(). Initializing PolicyNetwork from checkpoint.
2020-08-04 06:20:18.255 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 51: [loading model from /content/dp_big_demo_dir/model/policy]


INFO:tensorflow:Restoring parameters from /content/dp_big_demo_dir/model/policy


In [28]:
bot.reset()

bot(["Hey!"])[0][0].actions_tuple

('utter_hi',)

Ok the bot performs OK on greeting

In [29]:
bot(["I'd like to find a restaurant for this evening"])[0][0].actions_tuple

('utter_OFFER_RestaurantName', 'OFFER_City')

The bot asks for an inevitably necessary information.

In [30]:
bot(["Somewhere in Oakland, sushi and for two people please"])[0][0].actions_tuple

('utter_OFFER_RestaurantName', 'OFFER_City')

And so on.

In [31]:
bot(["Cool! That's what I was looking for, thanks!"])[0][0].actions_tuple

('utter_CONFIRM_RestaurantName',
 'CONFIRM_City',
 'CONFIRM_Time',
 'CONFIRM_PartySize',
 'CONFIRM_Date')

Let's say goodbye to our bot

In [32]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_CONFIRM_Time', 'CONFIRM_Date')

While it'd be nice for it to reply "Good bye!", it didn't. Why? Given that the DSTC dataset doesn't support this utterance, our bot can't properly react to user's response. This can be easily fixed by re-using the "Bye" intent used in the Basic Demo above.

You will have to add it to all 3 files (note that in case of `domain.yml` you have to add one line to *intents* and another two to *responses* sections of the file):

#### stories.md
```
## goodbye
* bye
  - utter_bye
```

#### nlu.md
```
## intent:bye
- goodbye
- goodnight
- good bye
- good night
- see ya
- toodle-oo
- bye bye
- gotta go
- farewell
```

#### domain.yml
```
intents:
  - bye

responses:
  utter_bye:
  - text: Bye!
```

You will also have to re-use our stories.md for train, test, and valid nlu.md files. Again, for the purposes of this demo, we use the same files for stories.md.

In [ ]:
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-trn.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-tst.md
!cp schema_resto_md_yaml/stories.md schema_resto_md_yaml/stories-val.md

Re-training our bot:

In [ ]:
from deeppavlov import train_model

train_model(gobot_config, download=False);

Checking it:

In [33]:
bot(["Bye bot"])[0][0].actions_tuple

('utter_CONFIRM_RestaurantName',
 'CONFIRM_City',
 'CONFIRM_Time',
 'CONFIRM_PartySize',
 'CONFIRM_Date')

## Final words

More of in-depth examples of goal-oriented bots and their features are coming with future releases, stay tuned!